# C1 W4 Group 8 - Task 4


In [1]:
from PIL import Image

from src.data import AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST
from src.paths import BBDD_PATH, QST1_W4_PATH, WEEK_4_RESULTS_PATH
from tqdm import tqdm

In [2]:
BBDD_PATH_LIST = sorted(BBDD_PATH.glob("*.jpg"))
QST1_W4_PATH_LIST = sorted(QST1_W4_PATH.glob("*.jpg"))

In [3]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in BBDD_PATH_LIST]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [4]:
query_d1_image_PIL_list = [Image.open(query_img_path) for query_img_path in QST1_W4_PATH_LIST]  # Load once
for idx, query_img in enumerate(query_d1_image_PIL_list):
    assert query_img.filename.endswith(f"{idx}.jpg")

In [5]:
for file, augmentation, frames, gt in zip(QST1_W4_PATH_LIST, AUGMENTATIONS_QSD1_W4_LIST, FRAMES_QSD1_W4_LIST, GT_QSD1_W4_LIST):
    print(f"File={file.stem}, Aug={augmentation}, Frames={frames}, GT={gt}")

File=00000, Aug=None, Frames=[[np.float64(179.49631381503272), [[64, 41], [2048, 25], [2016, 3653], [196, 3637]]]], GT=[-1]
File=00001, Aug=None-MultiplyHue, Frames=[[np.float64(179.52154949700142), [[19, 38], [509, 37], [496, 665], [17, 661]]]], GT=[150]
File=00002, Aug=None-MultiplyHue, Frames=[[np.float64(0.9495668564358084), [[34, 50], [396, 32], [408, 470], [46, 476]]], [np.float64(0.3015539498682642), [[466, 34], [844, 20], [850, 484], [470, 486]]]], GT=[48, 251]
File=00003, Aug=UnnamedImpulseNoise, Frames=[[np.float64(180.0), [[24, 27], [458, 26], [462, 355], [45, 355]]]], GT=[32]
File=00004, Aug=None-MultiplyHue, Frames=[[np.float64(0.4841781604809512), [[53, 78], [1489, 62], [1497, 2326], [77, 2338]]]], GT=[161]
File=00005, Aug=None, Frames=[[np.float64(179.85346379900508), [[50, 43], [1642, 27], [1646, 2051], [82, 2047]]]], GT=[81]
File=00006, Aug=None, Frames=[[np.float64(1.5481576989779455), [[176, 46], [470, 34], [478, 326], [182, 334]]], [np.float64(1.3510650827196287), [

## Background Removal

In [6]:
from src.background import get_painting_masks, crop_image_by_mask

In [7]:
painting_masks_list_d1 = []
pbar = tqdm(zip(QST1_W4_PATH_LIST, query_d1_image_PIL_list))
for name, image in pbar:
    pbar.set_description(f"Processing {name.stem}")
    painting_masks = get_painting_masks(image)
    painting_masks_list_d1.append(painting_masks)

Processing 00029: : 30it [00:20,  1.50it/s]


In [8]:
cropped_query_image_list_d1 = []
pbar = tqdm(zip(QST1_W4_PATH_LIST, query_d1_image_PIL_list, painting_masks_list_d1))
for name, image, predicted_mask in pbar:
    pbar.set_description(f"Splitting {name.stem}")
    image_list = crop_image_by_mask(image, predicted_mask)
    cropped_query_image_list_d1.append(image_list)

Splitting 00029: : 30it [00:01, 28.57it/s]


## Image Denoising

In [9]:
from src.denoising import denoise_image

In [10]:
denoised_query_d1_PIL_list = []
for image_tuple in cropped_query_image_list_d1:
    temp = []
    for img in image_tuple:
        temp.append(denoise_image(img))
    denoised_query_d1_PIL_list.append(temp)

## Retrieval

In [11]:
from src.descriptors import ImageRetrievalSystem, SIFTDescriptor, HOGDescriptor, ORBDescriptor

In [12]:
descriptors = {
    'SIFT': SIFTDescriptor(max_features=500),
    'HOG13': HOGDescriptor(pixels_per_cell=12, cells_per_block=4),
    'ORB': ORBDescriptor(),
}
descriptor_path = WEEK_4_RESULTS_PATH / 'Task_1' / 'descriptors'  

retrieval_system = ImageRetrievalSystem(descriptors, descriptor_path, log=False)

In [13]:
#Set number of top similar images to retrieve
K = 10

results = {}
g_0 = {}

descriptor_name = "HOG13"

results[descriptor_name] = retrieval_system.retrieve_similar_images(descriptor_name, database_image_PIL_list, denoised_query_d1_PIL_list, None, K)

287it [00:00, 36150.19it/s]
287it [00:00, 42835.57it/s]
287it [00:00, 43816.30it/s]
287it [00:00, 43584.68it/s]
287it [00:00, 43676.40it/s]
287it [00:00, 36578.60it/s]
287it [00:00, 40262.40it/s]
287it [00:00, 43196.80it/s]
287it [00:00, 39904.70it/s]
287it [00:00, 39444.43it/s]
287it [00:00, 39487.13it/s]
287it [00:00, 39272.00it/s]
287it [00:00, 41751.01it/s]
287it [00:00, 40112.14it/s]
287it [00:00, 40645.77it/s]
287it [00:00, 42057.34it/s]
287it [00:00, 40012.14it/s]
287it [00:00, 41082.74it/s]
287it [00:00, 42210.72it/s]
287it [00:00, 34498.76it/s]
287it [00:00, 41254.51it/s]
287it [00:00, 40732.42it/s]
287it [00:00, 36384.02it/s]
287it [00:00, 41140.30it/s]
287it [00:00, 40993.20it/s]
287it [00:00, 39470.30it/s]
287it [00:00, 38389.04it/s]
287it [00:00, 39369.61it/s]
287it [00:00, 42511.84it/s]
287it [00:00, 41011.35it/s]
287it [00:00, 42076.45it/s]
287it [00:00, 40530.82it/s]
287it [00:00, 40779.34it/s]
287it [00:00, 41569.35it/s]
287it [00:00, 41563.61it/s]
287it [00:00, 40954.

In [14]:
print(results[descriptor_name])

[[[223, 218, 237, 215, 133, 63, 259, 225, 57, 132]], [[35, 11, 105, 88, 174, 93, 140, 21, 46, 92]], [[203, 124, 11, 238, 40, 177, 16, 161, 197, 53], [211, 222, 40, 193, 113, 104, 177, 196, 84, 286]], [[176, 210, 214, 255, 191, 40, 282, 194, 245, 54]], [[259, 215, 174, 60, 184, 163, 271, 29, 104, 113]], [[-1]], [[-1]], [[-1]], [[-1]], [[258, 104, 163, 174, 60, 259, 215, 119, 11, 88], [-1]], [[233, 21, 191, 57, 3, 74, 92, 40, 282, 11], [252, 215, 237, 57, 259, 51, 58, 29, 133, 225]], [[-1]], [[-1]], [[218, 160, 133, 57, 40, 237, 244, 58, 137, 135]], [[-1]], [[93, 197, 46, 198, 274, 78, 156, 11, 21, 74]], [[94, 174, 32, 234, 140, 127, 105, 282, 104, 21], [132, 137, 63, 237, 104, 271, 60, 174, 29, 11]], [[11, 238, 282, 92, 180, 40, 21, 54, 35, 93]], [[-1]], [[9, 174, 259, 60, 29, 88, 104, 23, 239, 140]], [[237, 218, 135, 29, 281, 37, 132, 45, 277, 60], [-1]], [[-1]], [[83, 11, 238, 124, 92, 196, 152, 254, 197, 138]], [[-1]], [[104, 40, 174, 32, 11, 140, 193, 282, 88, 244]], [[23, 174, 105,